In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip

In [ ]:
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms , models
from tqdm.notebook import tqdm

In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True  

In [ ]:
seed = 24
seed_everything(seed)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
#load data
train_dir = './train/'
test_dir = './test/'

In [ ]:
from glob import glob
train_list = glob(os.path.join(train_dir,'*'))
test_list = glob(os.path.join(test_dir,'*'))
print(f"Num of train {len(train_list)}")
print(f"Num of test {len(test_list)}")

In [ ]:
train_labels = [path.split('/')[-1].split('.')[0] for path in train_list]
test_labels = [path.split('/')[-1].split('.')[0] for path in test_list]
print(len(train_labels),len(test_labels))

In [ ]:
count = 0
for it in train_labels:
  if it =='cat':
    count += 1
print(f"num of cat {count}")
print(f"num of dog {len(train_labels) - count}")

In [ ]:
#plot some image
random_idx = np.random.randint(1, len(train_list),size=9)
fig, axes = plt.subplots(3, 3, figsize=(7,5))
for idx, ax in enumerate(axes.ravel()):
  img = Image.open(train_list[idx])
  print(img.size)
  ax.set_title(train_labels[idx])
  ax.imshow(img)

In [ ]:
#split data
list_train , list_valid = train_test_split(train_list, test_size = 0.2, stratify = train_labels,random_state=seed)
print(f'Num of train: {len(list_train)}')
print(f'Num of valid: {len(list_valid)}')

In [ ]:
#Image augmentation
image_size = (224,224)
train_transforms = transforms.Compose(
    [
     transforms.Resize(image_size),
     transforms.RandomResizedCrop(image_size),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
    ]
)

valid_transforms = transforms.Compose(
    [
     transforms.Resize(image_size),
     transforms.RandomResizedCrop(image_size),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
    ]
)

test_transforms = transforms.Compose(
    [
     transforms.Resize(image_size),
     transforms.RandomResizedCrop(image_size),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
    ]
)

In [ ]:
#class Dataset
class CustomDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        if img_path.split('/')[-1][-3:] == 'jpg':
            img = Image.open(img_path)     
            if self.transform is not None:
                img_transform = self.transform(img)
                label = img_path.split('/')[-1].split('.')[0]
                label = 1 if label == 'dog' else 0
        return img_transform, label

In [ ]:
train_data = CustomDataset(list_train, transform=train_transforms)
valid_data = CustomDataset(list_valid, transform=valid_transforms)
test_data = CustomDataset(test_list, transform=test_transforms)

In [ ]:
batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
print(len(train_data),len(train_loader))
print(len(valid_data),len(valid_loader))

In [ ]:
#model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

In [ ]:
lr = 3e-5
gamma = 0.2
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
%%time
epochs = 20
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )
